# wandb api를 사용하여 최적 모델 저장하기

In [18]:

import argparse
import sys
import shutil
import json
from pathlib import Path
import tempfile
from typing import Optional, Union
import wandb
from IPython.display import Image

In [8]:
# 현재 주피터 노트북 파일 경로
file_path = r"C:\Users\ftmlab\Documents\hyoon\side_project\code_note\wandb\api.ipynb"

In [71]:
FILE_NAME = Path(file_path).resolve()

# 최적 모델을 저장할 경로,
# 로컬에서 로그가 저장되어 있는 루트 경로 설정
SAVED_MODEL_DIRNAME = FILE_NAME.parents[0] / "saved_model"
TRAINING_LOGS_DIRNAME = FILE_NAME.parent / "logs"

entity: userid    
project: 프로젝트 이름    
train_data_class: 훈련에 사용한 데이터 클래스    
metric: 최적 모델을 정할 기준 metric    
mode: metric의 최적을 정하는 기준    

entity와 project 이름은 wandb 웹에 접속한 후 run의 overview 페이지에서 "run path"를 보면 확인할 수 있다.
"Run path" 의 포멧은 다음과 같다.    
`<entity>/<project>/<run_id>`    
![wandb_overview](".\images\wandb_overview_run_path.PNG")

In [28]:
entity = "hgyoon0928"
project = "Full-stack-deeplearning-LABs_lab5_training"
trained_data_class = "EMNISTLines2"
metric = "val_loss"
mode = "min"

## Run 불러오고 정렬하기

mode에 따라 내림차순으로 정렬할지 오른차순으로 정렬할지 정한다.    
data_class를 사용한 run들을 matric 값으로 정렬한 후 맨 앞 run 정보를 가져오는 방식으로 최적의 run을 가져온다.

In [16]:
if mode == "min":
    default_metric_value = sys.maxsize
    sort_reverse = False
else:
    default_metric_value = 0
    sort_reverse = True

1. wandb apl 인스턴스를 받은 후 config의 trained_data_class에 해당하는 run들만 추출한다.
2. runs을 metric 값으로 정렬한 후 가장 앞 run(최적 모델)을 받는다.

![wandb_overview_config](images\wandb_overview_config.PNG")

In [23]:
def _get_summary_value(wandb_run: wandb.apis.public.Run, key: str, default: int) -> Union[int, float]:
    """
       key에 해당하는 수치 값(summary[key])을 반환한다. 값이 타당한 자료형이 아니면 디폴트 값을 반환한다.
    """
    value = wandb_run.summary.get(key, default)
    if not isinstance(value, (int, float)):
        value = default
    return value

In [29]:
api = wandb.Api()
# filter에 따라 runs 불러오기
runs = api.runs(f"{entity}/{project}", filters={"config.data_class": trained_data_class})
# metric으로 runs 정렬
sorted_runs = sorted(
        runs,
        key=lambda run: _get_summary_value(wandb_run=run, key=metric, default=default_metric_value),
        reverse=sort_reverse,
    )
best_run = sorted_runs[0]
summary = best_run.summary
# name, id
print(f"Best run ({best_run.name}, {best_run.id}) picked from {len(runs)} runs with the following metrics:")
print(f" - val_loss: {summary['val_loss']}, val_cer: {summary['val_cer']}, test_cer: {summary['test_cer']}")

Best run (polar-sweep-4, yolbb6ft) picked from 39 runs with the following metrics:
 - val_loss: 0.15537147223949432, val_cer: 0.05063715577125549, test_cer: 0.04524993523955345


## summary와 config 정보 가져오기

각 정보는 딕셔너리처럼 사용 가능하다.

In [45]:
print(type(runs[0].config))
print(type(runs[0].summary))

<class 'dict'>
<class 'wandb.old.summary.HTTPSummary'>


In [39]:
runs[0].summary.get("val_loss", default_metric_value)

'NaN'

In [41]:
runs[0].config

{'lr': 0.01,
 'gpus': 1,
 'loss': 'transformer',
 'wandb': True,
 'fc_dim': 512,
 'logger': True,
 'tf_dim': 64,
 'plugins': 'None',
 'conv_dim': 64,
 'profiler': 'None',
 'tf_nhead': 8,
 'amp_level': 'O2',
 'benchmark': False,
 'max_steps': 'None',
 'min_steps': 'None',
 'num_nodes': 1,
 'optimizer': 'Adam',
 'precision': 16,
 'tf_fc_dim': 128,
 'tf_layers': 4,
 'tpu_cores': '_gpus_arg_default',
 'batch_size': 32,
 'data_class': 'EMNISTLines2',
 'max_epochs': 'None',
 'max_length': 43,
 'min_epochs': 'None',
 'tf_dropout': 0.4,
 'accelerator': 'None',
 'amp_backend': 'native',
 'max_overlap': 0.5,
 'min_overlap': 0.2,
 'model_class': 'LineCNNTransformer',
 'num_workers': 1,
 'augment_data': 'true',
 'auto_lr_find': False,
 'fast_dev_run': False,
 'window_width': 16,
 'deterministic': False,
 'num_processes': 1,
 'window_stride': 8,
 'log_gpu_memory': 'None',
 'sync_batchnorm': False,
 'load_checkpoint': 'None',
 'overfit_batches': 0,
 'track_grad_norm': -1,
 'weights_summary': 'top',


## wandb 서버에서 불러와서 저장하기

In [59]:
run_for_download_ckpt = [run for run in runs if run.name == "lucky-salad-3"][0]

In [60]:
checkpoint_wandb_files = [file for file in run_for_download_ckpt.files() if file.name.endswith(".ckpt")]

In [61]:
checkpoint_wandb_files # 없으면 빈 리스트 반환

[<File training/logs/Full-stack-deeplearning-LABs_lab5_training/1e1oluxs/checkpoints/epoch=009-val_loss=2.499-val_cer=0.770.ckpt () 54.2MiB>]

In [64]:
wandb_file = checkpoint_wandb_files[0]
wandb_file

<File training/logs/Full-stack-deeplearning-LABs_lab5_training/1e1oluxs/checkpoints/epoch=009-val_loss=2.499-val_cer=0.770.ckpt () 54.2MiB>

In [65]:
wandb_file.name

'training/logs/Full-stack-deeplearning-LABs_lab5_training/1e1oluxs/checkpoints/epoch=009-val_loss=2.499-val_cer=0.770.ckpt'

In [74]:
wandb_file = checkpoint_wandb_files[0]
with tempfile.TemporaryDirectory() as tmp_dirname:
    wandb_file.download(root=tmp_dirname, replace=True)
    checkpoint_filename = f"{tmp_dirname}/{wandb_file.name}"
    shutil.copyfile(src=checkpoint_filename, dst=SAVED_MODEL_DIRNAME / "model.pt")
    print("Model checkpoint downloaded from wandb")
print(SAVED_MODEL_DIRNAME / "model.pt")

Model checkpoint downloaded from wandb
C:\Users\ftmlab\Documents\hyoon\side_project\code_note\wandb\saved_model\model.pt
